In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras.models as M
import tensorflow.keras.layers as L
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding
from IPython.display import clear_output
import json
import matplotlib.pyplot as plt

In [ ]:
#TF Setup for memory control
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
batch_size = 16
seq_length = 64

In [ ]:
def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / batch_size)
    
    for start in range(0, batch_chars - seq_length, 64):
        X = np.zeros((batch_size, seq_length))
        Y = np.zeros((batch_size, seq_length, unique_chars))
        for batch_index in range(0, 16):
            for i in range(0, 64):
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1
        yield X, Y

In [ ]:
def create_model():
    model = M.Sequential()
    model.add(L.Embedding(input_dim = 87, output_dim = 512, batch_input_shape = (batch_size, seq_length))) 
    model.add(L.LSTM(256, return_sequences = True, stateful = True))
    model.add(L.Dropout(0.1))
    model.add(L.LSTM(256, return_sequences = True, stateful = True))
    model.add(L.Dropout(0.1))
    model.add(L.LSTM(256, return_sequences = True, stateful = True))
    model.add(L.Dropout(0.1))
    model.add(L.TimeDistributed(Dense(87)))
    model.add(L.Activation("softmax"))
    return model

In [ ]:
def train(data,load=False,epochs=100):
    with open('char_to_index.json') as json_file:
        char_to_index = json.load(json_file)
    index_to_char = {i: ch for (ch, i) in char_to_index.items()}
    model = M.Sequential()
    model.add(L.Embedding(input_dim = 87, output_dim = 512, batch_input_shape = (batch_size, seq_length))) 
    model.add(L.LSTM(256, return_sequences = True, stateful = True))
    model.add(L.Dropout(0.1))
    model.add(L.LSTM(256, return_sequences = True, stateful = True))
    model.add(L.Dropout(0.1))
    model.add(L.LSTM(256, return_sequences = True, stateful = True))
    model.add(L.Dropout(0.1))
    model.add(L.TimeDistributed(Dense(87)))
    model.add(L.Activation("softmax"))
    if load:
        model.load_weights("weights/weights.h5", by_name=True)
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    loss, accuracy = [], []
    all_characters = np.asarray([char_to_index[c] for c in data], dtype = np.int32)
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        final_epoch_loss, final_epoch_accuracy = 0, 0    
        for i, (x, y) in enumerate(read_batches(all_characters, 87)):
            final_epoch_loss, final_epoch_accuracy = model.train_on_batch(x, y)
        clear_output()
        loss.append(final_epoch_loss)
        accuracy.append(final_epoch_accuracy)
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=[20,8])
        ax1.plot(loss, 'b')
        ax1.set_xlabel("epoch")
        ax1.set_ylabel("loss")
        ax2.plot(accuracy, 'r')
        ax2.set_xlabel("epoch")
        ax2.set_ylabel("accuracy")
        plt.show()
    model.save_weights("weights/weight_untrained.h5")

In [ ]:
file = open('dataset/dataset.txt', mode = 'r')
data = file.read()
file.close()
train(data,epochs=1)

In [ ]:
unique_chars = 87
def generate_music(initial, length):
    with open('char_to_index.json') as json_file:
        char_to_index = json.load(json_file)
    index_to_char = {i:ch for ch, i in char_to_index.items()}
    model = Sequential()
    model.add(Embedding(input_dim = 87, output_dim = 512, batch_input_shape = (1, 1))) 
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    model.add(LSTM(256, stateful = True))
    model.add(Dropout(0.2))
    model.add((Dense(87)))
    model.add(Activation("softmax"))
    model.load_weights("weights/weights.h5")
    sequence_index = [initial]
    
    for _ in range(length):
        batch = np.zeros((1, 1))
        batch[0, 0] = sequence_index[-1]
        
        predicted_probs = model.predict_on_batch(batch).ravel()
        sample = np.random.choice(range(unique_chars), size = 1, p = predicted_probs)
        
        sequence_index.append(sample[0])
    
    seq = ''.join(index_to_char[c] for c in sequence_index)
    
    cnt = 0
    for i in seq:
        cnt += 1
        if i == "\n":
            break
    seq1 = seq[cnt:]
    cnt = 0
    for i in seq1:
        cnt += 1
        if i == "\n" and seq1[cnt] == "\n":
            break
    seq2 = seq1[:cnt]
    return seq2

In [ ]:
generate_music(47,1000)